In [44]:
!git clone https://github.com/jorgpg5/synthetic_data.git

fatal: destination path 'synthetic_data' already exists and is not an empty directory.


# Synthesize Time Series data from your own DataFrame

This Blueprint demonstrates how to create synthetic time series data with Gretel. We assume that within the dataset
there is at least:

1) A specific column holding time data points

2) One or more columns that contain measurements or numerical observations for each point in time.

For this Blueprint, we will generate a very simple sine wave as our time series data.

In [45]:
%%capture

!pip install pyyaml smart_open numpy pandas optuna matplotlib
!pip install -U gretel-client

In [46]:
# Specify your Gretel API key

from getpass import getpass
import pandas as pd
from gretel_client import configure_session, ClientConfig

pd.set_option('max_colwidth', None)

configure_session(ClientConfig(api_key=getpass(prompt="Enter Gretel API key"), 
                               endpoint="https://api.gretel.cloud"))

Enter Gretel API key··········


In [47]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_df = pd.read_csv('/content/synthetic_data/dp_1.csv')

train_df

,Timestamp,BlinkL,BlinkR,Close_car,Dist_laneL,Dist_laneR,Ego_speed,FCW,Gral_pedestrian,GPS_data_type,GPS_timestamp,Validity,Lattitude,North_South,Longitude,East_West,Speed,Course,Date_stamp,Variation,East_West_variation,Checksum,Headway,Accel_X,Accel_Y,Accel_Z,Lane_curvature,Lane_heading,Marker,Number_obstacles,Obstacle_lane0,Obstacle_lane1,Obstacle_lane2,Obstacle_lane3,Obstacle_lane4,Obstacle_lane5,Obstacle_lane6,Obstacle_lane7,Obstacle_lane8,Obstacle_lane9,...,Left_wrist_x,Left_wrist_y,Left_wrist_z,Left_wrist_vis,Right_wrist_x,Right_wrist_y,Right_wrist_z,Right_wrist_vis,Left_pinky_x,Left_pinky_y,Left_pinky_z,Left_pinky_vis,Right_pinky_x,Right_pinky_y,Right_pinky_z,Right_pinky_vis,Left_index_x,Left_index_y,Left_index_z,Left_index_vis,Right_index_x,Right_index_y,Right_index_z,Right_index_vis,Left_thumb_x,Left_thumb_y,Left_thumb_z,Left_thumb_vis,Right_thumb_x,Right_thumb_y,Right_thumb_z,Right_thumb_vis,Left_hip_x,Left_hip_y,Left_hip_z,Left_hip_vis,Right_hip_x,Right_hip_y,Right_hip_z,Right_hip_vis
0,1620700292389898,0,0,0,0.00,0.00,0,0,0,$GPRMC,22428.6,A,2727.1414,S,15301.0568,E,0.00,307.79,110521,0.0,0.0,D*7F,0,0.00,1.75,-1.06,0.000000,0.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.464759,0.554156,-0.220379,0.984041,0.390232,0.636994,0.150019,0.599422,0.489942,0.643319,-0.000942,0.969260,0.403097,0.692561,-0.003690,0.575255,0.455006,0.655465,0.003837,0.973163,0.386248,0.727583,-0.001010,0.598732,0.443631,0.623236,0.000748,0.973681,0.381718,0.694005,-0.002471,0.622854,0.469241,0.158402,0.094456,0.996623,0.450585,0.290386,0.277914,0.998372
1,1620700292444904,0,0,0,-1.88,1.88,0,0,0,$GPRMC,23132.0,A,2727.1416,S,15301.0563,E,0.00,307.79,110521,0.0,0.0,D*7F,0,0.06,1.63,-1.25,-0.124846,-1.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.462843,0.548313,-0.218164,0.981558,0.394875,0.631561,0.104885,0.625000,0.488790,0.635836,-0.001074,0.964556,0.408985,0.686096,-0.003829,0.617666,0.454086,0.647278,0.003829,0.968933,0.389834,0.719757,-0.001213,0.644298,0.442736,0.615573,0.000748,0.970144,0.384171,0.686769,-0.002730,0.671047,0.476152,0.174349,0.070491,0.994896,0.457167,0.305354,0.257861,0.997148
2,1620700292520406,0,0,0,-1.88,1.88,0,0,0,$GPRMC,23132.0,A,2727.1416,S,15301.0563,E,0.00,307.79,110521,0.0,0.0,D*7F,0,0.06,1.63,-1.25,-0.124846,-1.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.463448,0.544341,-0.225338,0.983552,0.397095,0.635011,0.119700,0.572533,0.489134,0.629548,-0.001085,0.965311,0.414031,0.687534,-0.003845,0.533354,0.454693,0.642451,0.003839,0.969291,0.396106,0.719778,-0.001230,0.559251,0.443356,0.612133,0.000745,0.971142,0.390080,0.686804,-0.002738,0.590319,0.474365,0.167934,0.084372,0.997750,0.453799,0.303305,0.273701,0.998800
3,1620700292589380,0,0,0,-1.88,1.88,0,0,0,$GPRMC,23132.2,A,2727.1416,S,15301.0563,E,0.00,307.79,110521,0.0,0.0,D*7D,0,0.06,1.63,-1.25,-0.124846,-1.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.463570,0.538565,-0.219459,0.985744,0.394526,0.637846,0.127640,0.558906,0.489579,0.620777,-0.001112,0.969388,0.411822,0.690825,-0.003850,0.524997,0.455387,0.635648,0.003840,0.972776,0.394681,0.720904,-0.001306,0.550253,0.444100,0.606067,0.000727,0.974877,0.388989,0.688319,-0.002750,0.585817,0.475759,0.174586,0.081895,0.997823,0.453397,0.304718,0.269305,0.998857
4,1620700292670665,0,0,0,-1.88,1.88,0,0,0,$GPRMC,23132.2,A,2727.1416,S,15301.0563,E,0.00,307.79,110521,0.0,0.0,D*7D,0,0.06,1.63,-1.25,-0.124846,-1.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.462149,0.540912,-0.219192,0.982996,0.395914,0.635757,0.088897,0.670433,0.488676,0.621644,-0.001126,0.968004,0.413259,0.689328,-0.003858,0.648299,0.456415,0.637327,0.003848,0.971929,0.396848,0.718405,-0.001353,0.665301,0.445499,0.609764,0.000735,0.973119,0.391249,0.685648,-0.002744,0.688776,0.471799,0.170256,0.068155,0.991953,0.453873,0.308401,0.238535,0.996843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [48]:
all_cols = list(train_df.columns.values.tolist())
len(all_cols)

171

In [49]:
cols_to_remove = ['Timestamp', 'GPS_data_type', 'Validity', 'North_South', 'East_West', 'Checksum']
cols = all_cols.copy()
for elem in cols_to_remove:
  cols.remove(elem)
len(cols)

165

In [50]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=9, cols=1)

fig.append_trace(go.Scatter(
    y=train_df.Ego_speed,
    name='Ego_speed',
), row=1, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Lattitude,
    name='Lattitude',
), row=2, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Longitude,
    name='Longitude',
), row=3, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Accel_X,
    name='Accel_X',
), row=4, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Accel_Y,
    name='Accel_Y',
), row=5, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Accel_Z,
    name='Accel_Z',
), row=6, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Headway,
    name='Headway',
), row=7, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Number_obstacles,
    name='Number_obstacles',
), row=8, col=1)

fig.append_trace(go.Scatter(
    y=train_df.Left_elbow_x,
    name='Left_elbow_x',
), row=9, col=1)

fig.update_layout(height=1200, width=1200, title_text="Individual channels")
fig.show()

# Data As-Is

This part of the code generates synthetic data using the time series task with the data as-is. The hyperparameters we found that work best have been included for this run.

In [52]:
from smart_open import open
import yaml

from gretel_client import create_project
from gretel_client.helpers import poll

train_df = pd.read_csv('/content/synthetic_data/dp_1.csv')

# Create a project and model configuration.
project = create_project(display_name="time-series-synthetic-data-as-is")

# Pull down the default synthetic config.  We will modify it slightly.
with open("https://raw.githubusercontent.com/gretelai/gretel-blueprints/main/config_templates/gretel/synthetics/default.yml", 'r') as stream:
    config = yaml.safe_load(stream)

# Here we create an object to specify the timeseries task.
time_field = "Timestamp"
#trend_fields=["EDA", "ECG", "Left Pupil Diameter (m)", "Right Pupil Diameter (m)", "Eye Opening Left", "Eye Opening Right", "PERCLOS Value", "Blinking"]
#trend_fields = ['ECG'] #Used to see if training a single column does better than training all of them
trend_fields = cols

task = {
    'type': 'time_series',
    'attrs': {
        'time_field': time_field,
        'trend_fields': trend_fields
    }
}

config['models'][0]['synthetics']['task'] = task

config['models'][0]['synthetics']['params']['vocab_size'] = 0
config['models'][0]['synthetics']['params']['predict_batch_size'] = 1
config['models'][0]['synthetics']['params']['reset_states'] = True
config['models'][0]['synthetics']['params']['overwrite'] = True
config['models'][0]['synthetics']['params']['validation_split'] = False #Our validation split does not support time series tasks yet. Updated config will be uploaded soon.
config['models'][0]['synthetics']['params']['dropout_rate'] = .5
config['models'][0]['synthetics']['params']['gen_temp'] = .898
config['models'][0]['synthetics']['params']['learning_rate'] = .01 #0.0035
config['models'][0]['synthetics']['params']['rnn_units'] = 64

model = project.create_model_obj(model_config=config)

# Get a csv to work with, just dump out the train_df.
train_df.to_csv('train.csv', index=False)
model.data_source = 'train.csv'

# Upload the training data.  Train the model.
model.submit(upload_data_source=True)

poll(model)

# Use the model to generate synthetic data.
record_handler = model.create_record_handler_obj()

# For time series data we dump out the date column to seed the record handler.
train_df.Timestamp.to_csv('idx_seeds.csv', index=False)

record_handler.submit(
    action="generate",
    params={"num_records": 22600, "max_invalid": 20000},
    data_source='idx_seeds.csv',
    upload_data_source=True
)

poll(record_handler)

synthetic = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')

synthetic.head()

train_df.Ego_speed.head(1000).plot(figsize=(12, 8))
plt.show()

synthetic.Ego_speed.head(1000).plot(figsize=(12, 8))
plt.show()

INFO: Starting poller


{
    "uid": "61444f1373045fcf681604a3",
    "model_name": "festive-fluttering-lizard",
    "runner_mode": "cloud",
    "user_id": "60f7ffc2bff621796155eab8",
    "project_id": "61444f01ca06fb7b6606fee3",
    "logs": null,
    "status_history": {
        "created": "2021-09-17T08:17:23.421748Z"
    },
    "last_modified": "2021-09-17T08:17:23.475838Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/synthetics@sha256:8df2f21ed1b644eb3fc31ef1f924576e40b83f355634e4c62134722f5fd56d09",
    "model_type": "synthetics",
    "config": {
        "schema_version": "1.0",
        "name": null,
        "models": [
            {
                "synthetics": {
                    "params": {
                        "field_delimiter": null,
                        "epochs": 100,
                        "batch_size": 64,


INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2021-09-17T08:17:34.218376Z  Starting synthetic model training
2021-09-17T08:17:34.220619Z  Loading training data
2021-09-17T08:17:35.341233Z  Training data loaded
{
    "record_count": 22603,
    "field_count": 171
}
2021-09-17T08:17:48.890475Z  Creating semantic validators and preparing training data
2021-09-17T08:21:48.300738Z  Beginning ML model training
2021-09-17T08:22:17.147058Z  Training epoch completed
{
    "epoch": 0,
    "accuracy": 0.492,
    "loss": 1.4658,
    "val_accuracy": 0,
    "val_loss": 0,
    "batch": 0
}
2021-09-17T08:22:30.624055Z  Training epoch completed
{
    "epoch": 1,
    "accuracy": 0.5192,
    "loss": 1.346,
    "val_accuracy": 0,
    "val_loss": 0,
    "batch": 0
}
2021-09-17T08:22:44.071371Z  Training epoch completed
{
    "epoch": 2,
    "

ApiException: ignored

# Hyperparameter Tuning

Our lead machine learning researcher, Amy, found this fantastic library called Optuna to help us tune the configs for your use case. Feel free to take this code, and play with it to find new params! This helped us make sure that the errors you were seeing went away.

In [ ]:

def objective(trial: optuna.Trial):
    
    config['models'][0]['synthetics']['params']['vocab_size'] = trial.suggest_int(name="vocab_size", low=18, high=38, step=10)
    config['models'][0]['synthetics']['params']['rnn_units'] = trial.suggest_int(name="rnn_units", low=64, high=1024, step=64)
    config['models'][0]['synthetics']['params']['dropout_rate'] = trial.suggest_float("dropout_rate", .1, .75)
    config['models'][0]['synthetics']['params']['gen_temp'] = trial.suggest_float("gen_temp", .8, 1.2)
    config['models'][0]['synthetics']['params']['learning_rate'] = trial.suggest_float("learning_rate",  .001, 0.01, log=True)
    config['models'][0]['synthetics']['params']['reset_states'] = trial.suggest_categorical(
        "reset_states", choices=[True, False])
        
    seconds = int(time.time())
    project_name = "Tuning Experiment" + str(seconds)
    project = create_project(display_name=project_name)
    
    model = project.create_model_obj(model_config=config)

    # Get a csv to work with, just dump out the train_df.
    train_df.to_csv('train.csv', index=False)
    model.data_source = 'train.csv'

    # Upload the training data.  Train the model.
    model.submit(upload_data_source=True)

    status = "active"
    sqs = 0
    while status == "active":
        #Sleep a bit here
        time.sleep(60)
        models = []
        for model in project.search_models():
            ms = model.__dict__['_data']['model']
            ms = {key: ms[key] for key in ['model_name', 'model_type', 'status'] }
            status = ms["status"]
            print("Status is: " + status)
            if status == "completed":
                report = model.peek_report()
                if report:
                    sqs = report['synthetic_data_quality_score']['score']
                    print("Retrieved report sqs: " + str(sqs))
                else:
                    sqs = 0
            elif status == "error":
                sqs = 0
            
    return sqs

In [ ]:
# Create study that maximizes


study = optuna.create_study(direction="maximize")

# Start optimizing with however many trials you want
# You can do just one to see that it works, and then use the cells below
# to run more trials on the same study

study.optimize(objective, n_trials=50)

print(f"Optimized SQS: {study.best_value:.5f}")

print("Best params:")
for key, value in study.best_params.items():
    print(f"\t{key}: {value}")